# Converting the PDF to Excel

### first attempt, simple but too intutitive

In [1]:
import pandas as pd

# read txt
with open('castToTxt.txt', 'r') as file:
    lines = file.readlines()

# using a DF
data = []


# skip head line
for line in lines[1:]:
    parts = line.split(maxsplit=2)  # split into 3 parts: Ser, Category, SubCategory
    if len(parts) == 1:
        # only Ser, no Category or SubCategory
        ser = parts[0]
        category = None
        sub_category = None
    if len(parts) == 2: # SubCategory missing
        ser, category = parts
        sub_category = None
    elif len(parts) == 3:
        ser, category, sub_category = parts
    
    try:
        ser = int(ser)
    except ValueError:
        continue
    
    data.append([ser, category, sub_category])
    # data.append([int(ser), category, sub_category])

# add tp DF
df = pd.DataFrame(data, columns=['Ser', 'Category', 'SubCategory'])

# print df
df.head(15)

,Ser,Category,SubCategory
0,1,Agriculture,None
1,2,Agricultural,Equipment\n
2,3,None,None
3,4,None,None
4,5,None,None
5,6,None,None
6,7,None,None
7,8,None,None
8,9,None,None
9,10,None,None


In [2]:
df.to_excel('output.xlsx', index=False)

print("DataFrame saved to output.xlsx")

DataFrame saved to output.xlsx


    It looks like we have a skeleton, but a few issues exist, lets see if we can get subcategory to work

### Second attempt, Categories & Sub categories only

    we will start out with the first 1/2, spliting categories & subcategories 

In [3]:
import re # allows multiple words

with open('castToTxtFirst.txt', 'r') as file:
    lines = file.readlines()

# list of SER values that should be categories
category_ser_list = [
    1, 152, 341, 535, 749, 927, 1011, 1070, 1316, 1384, 1467, 1511, 1545, 1560, 
    1566, 1645, 1815, 1950, 1994, 2017, 2074, 2344, 2513, 2580, 2633, 3039, 
    3121, 3207, 3388, 3549, 3604, 3680, 3764, 3819, 3872, 4070, 4122, 4260, 
    4316, 4406, 4476
]

# category_ser_list = list(range(1, 4596))  # SER values 1 to 20 as categories

data = []
current_category = None

# # each line
for line in lines[1:]:
    
    line = line.strip()             # keeps empty lines
    if not line:  # check for empty lines
        data.append([None, None, None])
        continue
    
    match = re.match(r'(\d+)\s+(.*)', line)
    if not match:
        continue
    
    ser = int(match.group(1))
    sub_category = match.group(2)
    
    if ser in category_ser_list:
        current_category = sub_category
        data.append([ser, None, current_category])
    else:
        category = current_category
        data.append([ser, category, sub_category])

df = pd.DataFrame(data, columns=['Ser', 'Category', 'SubCategory'])


    This section of code makes sure empty columns for a future subsub category & subsubsub category are included

In [4]:
all_ser_values = list(range(1, 4596))

missing_data = {
    'Ser': [],
    'Category': [],
    'SubCategory': []
}

existing_sers = set(df['Ser'])
for ser in all_ser_values:
    if ser in existing_sers:
        continue  # skip if SER already exists
    
    missing_data['Ser'].append(ser)
    missing_data['Category'].append(None)
    missing_data['SubCategory'].append(None)

# append missing data & sort
missing_df = pd.DataFrame(missing_data)
df = pd.concat([df, missing_df], ignore_index=True)
df = df.sort_values(by='Ser')

    This section fixes the final formatting error (category names being misaligned)

In [5]:
category_ser_list = [
    1, 152, 341, 535, 749, 927, 1011, 1070, 1316, 1384, 1467, 1511, 1545, 1560, 
    1566, 1645, 1815, 1950, 1994, 2017, 2074, 2344, 2513, 2580, 2633, 3039, 
    3121, 3207, 3388, 3549, 3604, 3680, 3764, 3819, 3872, 4070, 4122, 4260, 
    4316, 4406, 4476
]

# update category names in DF
for index, row in df.iterrows():
    if row['Ser'] in category_ser_list:
        category_name = row['SubCategory']
        df.at[index, 'Category'] = category_name
        df.at[index, 'SubCategory'] = ''

    & finally this one commits it to Excel

In [6]:

df.to_excel('output2a.xlsx', index=False)

print("DataFrame saved to output.xlsx")

DataFrame saved to output.xlsx


    We have 1 more job left in this section, this was manually done by using the array of category SER's, we will need a similar array before section 3

In [7]:
filtered_values = pd.DataFrame(df)

for x in filtered_values.index:
    if filtered_values.loc[x, 'Category'] != None:
        filtered_values.drop(x, inplace = True)
filtered_values = filtered_values.drop(['Category', 'SubCategory'], axis=1)
filtered_values.values.flatten()

print(filtered_values)


       Ser
909      3
910      4
911      5
912      6
913      7
...    ...
4590  4591
4591  4592
4592  4593
4593  4594
4594  4595

[3686 rows x 1 columns]


### Third attempt, Sub Sub (Categories & Sub Categories)

    Now lets try again with the sub sub category

In [8]:
with open('castToTxtSecond.txt', 'r') as file:
    lines = file.readlines()

# using a DF
data = []
ser_counter = 1 

# skip head line
for line in lines[1:]:
    parts = line.split(maxsplit=2)  # split into 3 parts: Ser, Category, SubCategory
    
    if len(parts) == 1:
        # only Ser, no Category or SubCategory
        sub_category = parts[0]
        category = None
        sub_sub_category = None
    if len(parts) == 2: # SubCategory missing
        sub_category, sub_sub_category = parts
        category = None
    elif len(parts) == 3:
        category, sub_category, sub_sub_category = parts
    
    # try:
    #     ser = int(ser)
    # except ValueError:
    #     continue
    
    data.append([ser_counter, category, sub_category, sub_sub_category])
    ser_counter += 1

# add tp DF
df2 = pd.DataFrame(data, columns=['Ser', 'Category', 'SubCategory', 'SubSubCategory'])

# print df
df2.head(15)

,Ser,Category,SubCategory,SubSubCategory
0,1,None,Agricultural,Greenhouses
1,2,None,Aquaculture,Equipment
2,3,None,Aquaculture,Trap
3,4,None,Fishing,Float
4,5,None,Fishing,Net
5,6,None,Fishing,Rope
6,7,Other,Aquaculture,Equipment\n
7,8,None,Ear,Tag
8,9,None,Peat,None
9,10,None,Eggs,None


    The categorization is now split, & our ser is not supposed to be there, lets fix that

In [9]:
df2.fillna('', inplace=True)

df2['SubSubCategory'] = df2['Category'] + ' ' + df2['SubCategory'] + ' ' + df2['SubSubCategory']
df2['SubSubCategory'] = df2['SubSubCategory'].str.strip()
df2 = df2[['Ser', 'SubSubCategory']]

print(df2.head(15))

    Ser               SubSubCategory
0     1     Agricultural Greenhouses
1     2        Aquaculture Equipment
2     3             Aquaculture Trap
3     4                Fishing Float
4     5                  Fishing Net
5     6                 Fishing Rope
6     7  Other Aquaculture Equipment
7     8                      Ear Tag
8     9                         Peat
9    10                         Eggs
10   11             Fowl & Livestock
11   12                        Cacao
12   13                 Coffee Beans
13   14                Vanilla Beans
14   15                  Broad Beans


In [10]:
df2.drop(['Ser'], axis=1)

,SubSubCategory
0,Agricultural Greenhouses
1,Aquaculture Equipment
2,Aquaculture Trap
3,Fishing Float
4,Fishing Net
...,...
3618,Truck Steering
3619,Truck Suspension
3620,Truck Tire
3621,Truck Transmission


    Now lets take this to Excel

In [11]:
df2.to_excel('output2b.xlsx', index=False)

print("DataFrame 2b saved to output.xlsx")

DataFrame 2b saved to output.xlsx


    Next step is to spread df2 out

In [12]:
df3 = pd.DataFrame(df)

df3['SubSubCategory'] = df3.apply(lambda _: '', axis=1)

last_Cat = ""; last_Sub = ""; i = 0

for x in df3.index:
    last_Cat = df3.loc[x, 'Category']; last_Sub = df3.loc[x, 'SubCategory']
    if df3.loc[x, 'Category'] == None:
        df3.loc[x, 'Category'] = last_Cat
        df3.loc[x, 'SubCategory'] = last_Sub
        df3.loc[x, 'SubSubCategory'] = df2.loc[i, 'SubSubCategory']
        if i < 3622:
            i = i + 1

print(df3)

# last_Cat = df3.loc[21, 'Category']; last_Sub = df3.loc[x+1, 'SubCategory']

# print(last_Cat)

       Ser     Category             SubCategory            SubSubCategory
0        1  Agriculture                                                  
1        2  Agriculture  Agricultural Equipment                          
909      3         None                    None  Agricultural Greenhouses
910      4         None                    None     Aquaculture Equipment
911      5         None                    None          Aquaculture Trap
...    ...          ...                     ...                       ...
4590  4591         None                    None               Truck Wheel
4591  4592         None                    None               Truck Wheel
4592  4593         None                    None               Truck Wheel
4593  4594         None                    None               Truck Wheel
4594  4595         None                    None               Truck Wheel

[4595 rows x 4 columns]


    And its extraction time

In [13]:
df3.to_excel('output3.xlsx', index=False)

print("DataFrame 3 saved to output.xlsx")

DataFrame 3 saved to output.xlsx


### Fourth & final attempt, clean up:

    Now we will attempt the tree implementation

In [15]:
df4 = pd.DataFrame(df3)

df4.replace('', pd.NA, inplace=True)

# Apply forward fill to the 'Category' and 'SubCategory' columns
df4[['Category', 'SubCategory']] = df4[['Category', 'SubCategory']].ffill()

# Print the resulting DataFrame for verification
print(df4)

       Ser        Category             SubCategory            SubSubCategory
0        1     Agriculture                    <NA>                      <NA>
1        2     Agriculture  Agricultural Equipment                      <NA>
909      3     Agriculture  Agricultural Equipment  Agricultural Greenhouses
910      4     Agriculture  Agricultural Equipment     Aquaculture Equipment
911      5     Agriculture  Agricultural Equipment          Aquaculture Trap
...    ...             ...                     ...                       ...
4590  4591  Transportation             Truck Parts               Truck Wheel
4591  4592  Transportation             Truck Parts               Truck Wheel
4592  4593  Transportation             Truck Parts               Truck Wheel
4593  4594  Transportation             Truck Parts               Truck Wheel
4594  4595  Transportation             Truck Parts               Truck Wheel

[4595 rows x 4 columns]
